<a href="https://colab.research.google.com/github/kriskirla/AllProjects/blob/master/Algotrading/Portfolio_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Description: Analyze my portfolio

In [ ]:
!pip install PyPortfolioOpt
!pip install pulp
!pip install yfinance

In [2]:
# Import libraries 
import pandas as pd
import numpy as np
import requests
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime as dt

In [3]:
# Tickers to track S&P500
payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
second_table = payload[1]

df = first_table
ticker = df['Symbol'].values.tolist()

In [4]:
# Get information about ticker
def get_company_information(symbol):
  ticker = yf.Ticker(symbol)
  return ticker.info

#get_company_information("ZOM")

In [11]:
# Get data from yahoo finance
df = yf.download(ticker, start="2019-01-01", end=dt.today().strftime('%Y-%m-%d'))['Close']
# Drop all delisted tickers
df = df.dropna(axis='columns', how='all')
df = df.drop(columns=["WRK"]) # For some reason WRK is showing Na for all except latest
df

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,...,VFC,VIAC,VLO,VMC,VNO,VNT,VRSK,VRSN,VRTX,VTR,VTRS,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,65.690002,32.480000,157.919998,39.480000,89.230003,74.449997,309.959991,69.500000,140.589996,224.570007,85.919998,40.950001,130.139999,128.960007,63.529999,72.940002,14.180000,45.520000,39.619999,90.099998,71.889999,59.009998,78.110001,202.119995,61.520000,81.639999,81.050003,98.050003,33.480000,9.30,18.830000,67.489998,191.899994,107.959999,156.740005,1539.130005,210.550003,143.009995,255.020004,144.039993,...,67.184555,NaN,75.559998,97.589996,60.360001,NaN,108.650002,147.759995,164.080002,56.919998,27.920000,56.020000,70.860001,183.410004,68.040001,38.259998,67.580002,67.290001,46.939999,107.919998,149.679993,88.720001,22.420000,93.339996,48.279999,96.410004,17.139999,21.530001,104.400002,48.220001,86.849998,69.690002,37.889999,20.129999,66.510002,91.440002,102.279999,156.240005,41.580002,84.480003
2019-01-03,63.270000,30.059999,162.850006,35.547501,86.290001,73.370003,302.290009,66.220001,135.789993,215.699997,80.730003,40.770000,126.209999,121.849998,63.689999,72.769997,14.170000,44.369999,38.750000,88.660004,70.540001,57.700001,72.690002,184.779999,58.119999,80.300003,79.680000,100.209999,31.540001,9.30,17.049999,65.570000,188.979996,105.099998,157.679993,1500.280029,199.600006,137.639999,248.419998,141.880005,...,65.207153,NaN,74.199997,96.589996,60.200001,NaN,105.330002,142.589996,163.729996,57.880001,27.230000,56.220001,69.480003,176.339996,67.330002,35.650002,67.760002,68.529999,46.570000,107.559998,146.539993,88.480003,22.610001,92.860001,47.480000,94.529999,16.870001,22.260000,103.389999,48.029999,83.580002,68.620003,37.910000,19.690001,64.230003,89.139999,100.430000,146.880005,41.349998,81.559998
2019-01-04,65.459999,32.040001,158.809998,37.064999,89.070000,74.220001,313.440002,68.110001,141.070007,226.190002,82.690002,41.709999,131.240005,128.279999,64.589996,73.440002,14.640000,44.919998,40.180000,90.449997,72.320000,59.380001,76.239998,186.710007,60.000000,82.669998,79.500000,106.000000,33.700001,9.30,19.000000,68.050003,195.440002,110.730003,158.309998,1575.390015,202.470001,143.660004,249.910004,144.550003,...,67.448212,NaN,77.010002,101.529999,61.209999,NaN,109.980003,148.970001,172.699997,57.599998,28.840000,56.360001,71.250000,179.779999,69.570000,37.160000,68.260002,68.279999,47.950001,112.820000,150.500000,90.830002,23.549999,93.440002,48.213333,98.239998,17.090000,23.020000,107.559998,48.500000,87.449997,71.150002,39.009998,20.450001,66.940002,91.459999,103.860001,152.970001,42.619999,84.760002
2019-01-07,66.849998,32.950001,161.139999,36.982498,90.370003,74.750000,314.799988,69.129997,141.559998,229.259995,83.209999,41.919998,130.529999,132.720001,63.849998,73.029999,14.710000,45.160000,40.410000,90.809998,72.110001,61.240002,76.580002,189.919998,60.119999,82.540001,80.330002,107.940002,34.299999,9.50,20.570000,67.940002,198.070007,111.239998,159.289993,1629.510010,209.919998,147.320007,250.350006,144.229996,...,68.719398,NaN,77.089996,103.550003,62.380001,NaN,110.089996,151.399994,175.850006,57.919998,29.750000,56.720001,70.570000,187.240005,69.970001,38.480000,67.949997,69.089996,47.639999,116.110001,151.610001,91.050003,24.150000,94.540001,48.353333,98.550003,17.040001,23.480000,109.470001,48.290001,89.760002,71.519997,39.709999,20.8899

The Parts above is to input your portfolio

The parts below is to optimize portfolio

In [12]:
# Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [13]:
# Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for the maximal Sharpe ratio
ef = EfficientFrontier(mu, S) # Create the Efficient Frontier Object
weights = ef.max_sharpe()
clean_weights = ef.clean_weights()
print(clean_weights)
ef.portfolio_performance(verbose=True)

/usr/local/lib/python3.6/dist-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
/usr/local/lib/python3.6/dist-packages/pypfopt/risk_models.py:87: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn("Could not fix matrix. Please try a different risk model.")


OrderedDict([('A', 0.0), ('AAL', 0.0), ('AAP', 0.0), ('AAPL', 0.0), ('ABBV', 0.0), ('ABC', 0.0), ('ABMD', 0.0), ('ABT', 0.0), ('ACN', 0.0), ('ADBE', 0.0), ('ADI', 0.0), ('ADM', 0.0), ('ADP', 0.0), ('ADSK', 0.0), ('AEE', 0.0), ('AEP', 0.0), ('AES', 0.0), ('AFL', 0.0), ('AIG', 0.0), ('AIZ', 0.0), ('AJG', 0.0), ('AKAM', 0.0), ('ALB', 0.0), ('ALGN', 0.0), ('ALK', 0.0), ('ALL', 0.0), ('ALLE', 0.0), ('ALXN', 0.0), ('AMAT', 0.0), ('AMCR', 0.0), ('AMD', 0.0), ('AME', 0.0), ('AMGN', 0.0), ('AMP', 0.0), ('AMT', 0.0), ('AMZN', 0.0), ('ANET', 0.0), ('ANSS', 0.0), ('ANTM', 0.0), ('AON', 0.0), ('AOS', 0.0), ('APA', 0.0), ('APD', 0.0), ('APH', 0.0), ('APTV', 0.0), ('ARE', 0.0), ('ATO', 0.0), ('ATVI', 0.0), ('AVB', 0.0), ('AVGO', 0.0), ('AVY', 0.0), ('AWK', 0.0), ('AXP', 0.0), ('AZO', 0.0), ('BA', 0.0), ('BAC', 0.0), ('BAX', 0.0), ('BBY', 0.0), ('BDX', 0.0), ('BEN', 0.0), ('BIIB', 0.0), ('BIO', 0.0), ('BK', 0.0), ('BKNG', 0.0), ('BKR', 0.0), ('BLK', 0.0), ('BLL', 0.0), ('BMY', 0.0), ('BR', 0.0), ('BSX

/usr/local/lib/python3.6/dist-packages/cvxpy/problems/problem.py:1126: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


(2.7199856205636657, 0.4322335227226054, 6.246590045947075)

In [14]:
# Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

porfolio_val = 20000
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(clean_weights, latest_prices, porfolio_val)
allocation, leftover = da.lp_portfolio()
print(f"Discrete Allocation: {allocation}\nFunds Remaining: {leftover}")

Discrete Allocation: {'BEN': 1, 'CARR': 171, 'CLX': 10, 'ENPH': 34, 'TGT': 13, 'TSLA': 3, 'WST': 1}
Funds Remaining: 0.3701152801513672


In [15]:
# Save information to print
company_name = []
industry = []
share_price = []
total_price = []

for symbol in allocation:
    info = get_company_information(symbol)
    company_name.append(info['longName'])
    industry.append(info['industry'])
    share_price.append(info['regularMarketPrice'])
    total_price.append("${:.2f}".format(info['regularMarketPrice'] * allocation[symbol]))

In [16]:
# Print the final guestimation of what to buy
portfolio_df = pd.DataFrame(columns=["Ticker", "Name", "Allocation", "Price/Share", "Total", "Industry"])
portfolio_df["Ticker"] = allocation.keys()
portfolio_df["Name"] = company_name
portfolio_df["Allocation"] = allocation.values()
portfolio_df["Price/Share"] = share_price
portfolio_df["Total"] = total_price
portfolio_df["Industry"] = industry
portfolio_df

,Ticker,Name,Allocation,Price/Share,Total,Industry
0,BEN,"Franklin Resources, Inc.",1,27.92,$27.92,Asset Management
1,CARR,Carrier Global Corporation,171,37.23,$6366.33,Building Products & Equipment
2,CLX,The Clorox Company,10,182.22,$1822.20,Household & Personal Products
3,ENPH,"Enphase Energy, Inc.",34,190.02,$6460.68,Solar
4,TGT,Target Corporation,13,189.20,$2459.60,Discount Stores
5,TSLA,"Tesla, Inc.",3,779.09,$2337.27,Auto Manufacturers
6,WST,"West Pharmaceutical Services, Inc.",1,294.14,$294.14,Medical Instruments & Supplies
